# Disclamer

*This Jupiter notebook may be not fully reproducible.*

*I did not have a goal to make it reproducible*

*Just to show the way of my thoughts and their results*

# Research task

**Analyze the problem's data:** 

Review the step_title, step_description, the wrong_code_submission, and the correct_code_submission from the attached JSON files. The default_feedback represents the default response from Hyperskill's testing framework, and the gpt_current_feedback is the current AI-generated feedback. Use these as reference points for the improved feedback your solution will provide.

**Develop improved prompts:**   

Create new prompt(s) that would guide an AI model (like gpt-3.5-turbo or mixtral-small) to produce better feedback for the incorrect code submissions. Your prompts should aim to:
- Highlight the specific errors in the student's code.
- Offer actionable advice for correcting these errors.
- Encourage best coding practices without providing the solution directly.
- Consider the code_template and test_cases fields in your analysis to understand the initial setup and examples provided to the students, which may influence the context of their code submissions and the feedback needed.

# The approach:
1. **Research the dataset**
2. **Develop a prompt-building and evaluation system**
3. **Test a simple prompt**
4. **Make several attempts to improve it**
5. **Compare the results of the original and improved prompts** 
6. **Draw conclusions**


-----------------

# Importing libraries

In [98]:
from ipywidgets import HTML
import pandas as pd
import yaml
import markdown
import json

# Exploring data
Let's explore the datasets of python and java submissions

### Checking input data

In [92]:
with open('./input_data/python_code_problems_submissions.json', 'r') as f:
    python = json.load(f)
    
with open('./input_data/java_code_problems_submissions.json', 'r') as f:
    java = json.load(f)


python_df = pd.DataFrame(python['python']['code_problems'])
java_df = pd.DataFrame(java['java']['code_problems'])
print("Python df:")
display(python_df.head(2))
print("Java df:")
display(java_df.head(2))

Python df:


,topic_id,topic_title,topic_theory_url,step_id,step_title,step_description,step_url,code_template,test_cases,wrong_code_submission,default_feedback,gpt_current_feedback,correct_code_submission
0,399,Program with numbers,https://hyperskill.org/learn/step/5872,6464,Divide nuts equally between squirrels,<p></p><p>N squirrels found K nuts and decided...,https://hyperskill.org/learn/step/6464,# put your python code here,"[{'input': '3 14', 'output': '4'}]","# put your python code here\nN=int(input(""Ente...",Failed test #1 of 6. Wrong answer\n\nThis is a...,1. Ensure to separate inputs clearly by adding...,N = int(input())\nK = int(input())\nprint(K //...
1,534,"Function, arguments and parameters",https://hyperskill.org/learn/step/7248,10302,Equation,"<p></p>\n<p>Write a function <code class=""java...",https://hyperskill.org/learn/step/10302,def equation_writing(...):\n ...,"[{'input': '3 4 3', 'output': '3 x + 4 = 3'}]","def equation_writing(a, b, c):\n print(a, '...",Failed test #1 of 3. Wrong answer\n\nThis is a...,1. Change the existing print statement to use ...,"def equation_writing(a, b, c):\n print(f""{a..."


Java df:


,topic_id,topic_title,topic_theory_url,step_id,step_title,step_description,step_url,code_template,test_cases,wrong_code_submission,default_feedback,gpt_current_feedback,correct_code_submission
0,89,For-loop,https://hyperskill.org/learn/step/3505,2187,The sum of integers from a to b,<p>Print the sum of all integers from <strong>...,https://hyperskill.org/learn/step/2187,import java.util.Scanner;\n\nclass Main {\n ...,"[{'input': '5 21 ', 'output': '221'}]",import java.util.Scanner;\n\nclass Main {\n ...,Failed test #1 of 10. Runtime error\n\nThis is...,1. Correct the input reading: Ensure you read ...,import java.util.Scanner;\n\nclass Main {\n ...
1,7,Grouping classes with packages,https://hyperskill.org/learn/step/3621,9896,Sorting arrays,"<p></p>\n<p>The class <code class=""java"">Array...",https://hyperskill.org/learn/step/9896,class ArraySorting {\n /**\n * @param a...,"[{'input': 'y d b u t f m h x q', 'output': 'b...",class ArraySorting {\n /**\n * @param a...,Compilation error\nMain.java:17: error: incomp...,1. Change java.util.Arrays.sort(array) to java...,class ArraySorting {\n /**\n * @param a...


In [85]:
python_test_cases_count = len(df)
print("Python test cases")
python_test_cases_count

Python test cases


10

In [86]:
java_test_cases_count = len(df)
print("Java test cases")
java_test_cases_count

Java test cases


10

### Making render function

In [71]:
print("Submission response keys:")
list(df.columns)

Submission response keys:


['topic_id',
 'topic_title',
 'topic_theory_url',
 'step_id',
 'step_title',
 'step_description',
 'step_url',
 'code_template',
 'test_cases',
 'wrong_code_submission',
 'default_feedback',
 'gpt_current_feedback',
 'correct_code_submission']

In [305]:
def render_data(data, llm_feedback=None, passed=None, grading=[]):
    # Display topic title
    display(HTML(f"<h1>{data['topic_title']}</h1>"))

    # Display step title with link to the theory
    display(HTML(f"<h2><a href='{data['topic_theory_url']}'>{data['step_title']}</a></h2>"))

    # Display the description of the step
    display(HTML(data['step_description']))

    # Wrong Code submission
    display(HTML(f"<h3>Wrong Code Submission</h3><pre>{data['wrong_code_submission']}</pre>"))

    # Display test cases
    display(HTML("<h3>Test Cases</h3>"))
    for case in data['test_cases']:
        display(HTML(f"<strong>Input:</strong> {case['input']} <strong>Output:</strong> {case['output']}"))

    # Display feedback from a wrong submission
    display(HTML(f"<h3>Feedback for Wrong Submission</h3><pre>{data['default_feedback']}</pre>"))

    # Display feedback from GPT
    display(HTML(f"<h3>Additional GPT Feedback</h3><pre>{data['gpt_current_feedback']}</pre>"))

    # Display correct code submission
    display(HTML(f"<h3>Correct Code Submission</h3><pre>{data['correct_code_submission']}</pre>"))
    
    
    if llm_feedback:
        print("----------------------------------------------------")
        if 'User-Facing Feedback:' in llm_feedback:
            llm_feedback = markdown.markdown(llm_feedback.split('User-Facing Feedback:')[-1])
        display(HTML(f"<h3>GPT New Feedback (Passed: {passed})</h3><pre>{llm_feedback}</pre>"))
        display(HTML("<h3>Evaluation feedback:</h3>"))
        display(HTML(f"<h3>Score: {grading['score']}. Passed: {grading['pass']}</h3>"))
        display(HTML(f"<h3>Reason:</h3>"))
        print(grading['reason'] + "\n")

### Checking submission system response in human-readable form

In [163]:
step = 3
data = python['python']['code_problems'][step]
render_data(data)

HTML(value='<h1>Loop control: break, continue, pass</h1>')

HTML(value="<h2><a href='https://hyperskill.org/learn/step/6302'>Game over</a></h2>")

HTML(value='<p></p><p>In online test games, there is usually a limited number of lives: if, for example, you m…

HTML(value='<h3>Wrong Code Submission</h3><pre>scores = input().split()\n\n# put your python code here\nscore_…

HTML(value='<h3>Test Cases</h3>')

HTML(value='<strong>Input:</strong> C C C I C C C C I I C C C C C C C C C <strong>Output:</strong> Game over\n…

HTML(value='<h3>Feedback for Wrong Submission</h3><pre>Failed test #1 of 17. Runtime error\n\nThis is a sample…

HTML(value="<h3>Additional GPT Feedback</h3><pre>1. Change the comparison operator in the if statement where y…

HTML(value='<h3>Correct Code Submission</h3><pre>scores = input().split()\n# put your python code here\nlives …

# Making evaluation and test cases

Let's build assertion rules that will be used by another model (GPT-4) to test the prompt output

### Making test-cases

In [277]:
assertion_rules_1 = [
    "The user-faced feedback does not give direct answer",
    "The feedback is correct and highlight the specific errors in the student's code.",
    "The feedback offers actionable advice and encourages best coding practices.",
    "The feedback is easy to understand for a adult programming newbie with no experience."
]


def extract_important_data_1(data):
    return {
#         "topic": data['topic_title'],
#         "task_name": data['step_title'],
        "task_description": data['step_description'],
        "test_cases": data['test_cases'],
        "submission_system_output": data['default_feedback'],
#         "init_code_template": data['code_template'],
        "user_wrong_answer": data['wrong_code_submission'],
        "test_correct_answer": data['correct_code_submission'],
        # We don't need llm wrong feedback in testing system
        # "llm_wrong_feedback": data['gpt_current_feedback'],
    }

In [280]:
def make_testcase(prefix, id_, assertion_rules, extracted_data):
    return {
        "vars": {
            "id": f"{prefix}_{id_}", 
            **extracted_data
        },
        "assert": [
            {
                "type": "model-graded-closedqa",
                "value": rule,
            }
            for rule in assertion_rules
        ]
    }

def make_testcases(prefix, test_data, assertion_rules, extract_data):
    return [
        make_testcase(prefix, i, assertion_rules, extract_data(t))
        for i, t in enumerate(test_data)
    ]

# Writing data to a YAML file
with open('./tests/python.yaml', 'w') as file:
    python_test_cases = make_testcases(
        "python",
        test_data=python['python']['code_problems'],
        extract_data=extract_important_data_1,
        assertion_rules=assertion_rules_1,
    )
    yaml.dump(python_test_cases, file, default_flow_style=False, indent=2)
    
    
# Writing data to a YAML file
with open('./tests/java.yaml', 'w') as file:
    java_test_cases = make_testcases(
        "java",
        test_data=java['java']['code_problems'],
        extract_data=extract_important_data_1,
        assertion_rules=assertion_rules_1,
    )
    yaml.dump(java_test_cases, file, default_flow_style=False, indent=2)

### Making evaluation fn

In [1]:
def view_resulsts_table():
    !npx promptfoo@latest view -y

In [2]:
def make_eval(output_file_path):
    !npx promptfoo@latest eval -c ./promptfooconfig.yaml --no-progress-bar --output $output_file_path

# Testing the first prompt

In [ ]:
# Using template for each variable instead of json.dumps to be able to see input variables in evaluation results as table.
def create_data_template(data):
    return "\n" + "\n".join(map(lambda k: k + ": {{" + k + "}}", data.keys())) + "\n"

# Prompt is adapted for promptfoo evaluation system, so we are using {{}} instead of f-strings
prompt = \
f"""This is feedback of the coding testing system. 
Write a human feedback explaining the student what to fix
{create_data_template(extract_important_data_1(data))}
Highlight the specific errors in the student's code.
Offer actionable advice for correcting these errors.
Encourage best coding practices without providing the solution directly.
Make the answer simple to understand for a adult programming newbie with no experience."""

with open("./prompts/prompt.txt", 'w') as f:
    f.write(prompt)

In [259]:
make_eval("./outputs/output_1.json")


┌──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ id       │ submiss… │ task_de… │ test_ca… │ test_co… │ user_wr… │ This is  │
│          │          │          │          │          │          │ feedback │
│          │          │          │          │          │          │ of the   │
│          │          │          │          │          │          │ coding   │
│          │          │          │          │          │          │ testing  │
│          │          │          │          │          │          │ system.  │
│          │          │          │          │          │          │ Write a  │
│          │          │          │          │          │          │ human    │
│          │          │          │          │          │          │ feedback │
│          │          │          │          │          │          │ explain… │
│          │          │          │          │          │          │ the      │
│          │          │          │          │      

│          │ Error:   │          │          │          │          │ to call  │
│          │ Traceba… │          │          │          │          │ the      │
│          │ (most    │          │          │          │          │ function │
│          │ recent   │          │          │          │          │ twice as │
│          │ call     │          │          │          │          │ instruc… │
│          │ last):   │          │          │          │          │ in the   │
│          │   File   │          │          │          │          │ task...  │
│          │ "/sandb… │          │          │          │          │          │
├──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│ python_7 │ Failed   │ <p></p>  │ [{"inpu… │ word =   │ word =   │ [FAIL]   │
│          │ test #1  │ <p>The   │          │ input()  │ input()  │ The      │
│          │ of 8.    │ languag… │          │ # Change │ # Change │ submiss… │
│          │ Wrong    │ of the   │          │ the ne

» Run promptfoo view to use the local web viewer
» Run promptfoo share to create a shareable URL
» This project needs your feedback. What's one thing we can improve? https://forms.gle/YFLgTe1dKJKNSCsU7
Successes: 14
Failures: 6
Token usage: Total 76646, Prompt 0, Completion 0, Cached 9990
Done.


----

## FIRST ATTEMPT TOTAL SCORE: 70%

----

### Checking issues in the first prompt

In [260]:
with open("./outputs/output_1.json", 'r') as f:
    feedbacks = json.load(f)

def find_async_feedback_by_id(prefix, feedbacks, id_):
    for feedback in feedbacks['results']['results']:
        if feedback['vars']['id'] == f"{prefix}_{id_}":
            return feedback

In [306]:
step = 1
data = python['python']['code_problems'][step]
feedback = find_async_feedback_by_id("python", feedbacks, step)
llm_feedback = feedback['response']['output']
passed = feedback['success']
grading = feedback['gradingResult']
render_data(data, llm_feedback, passed, grading)

HTML(value='<h1>Function, arguments and parameters</h1>')

HTML(value="<h2><a href='https://hyperskill.org/learn/step/7248'>Equation</a></h2>")

HTML(value='<p></p>\n<p>Write a function <code class="java">equation_writing()</code> that will <strong>print<…

HTML(value="<h3>Wrong Code Submission</h3><pre>def equation_writing(a, b, c):\n    print(a, 'x +', b, '=', c)\…

HTML(value='<h3>Test Cases</h3>')

...

HTML(value='<h3>GPT New Feedback (Passed: False)</h3><pre>In your code, you are missing the f-string formattin…

HTML(value='<h3>Evaluation feedback:</h3>')

HTML(value='<h3>Score: 0.25. Passed: False</h3>')

HTML(value='<h3>Reason:</h3>')

The submission does not meet the criterion:
1. The criterion requires the feedback to be easy to understand for an adult programming newbie with no experience.
2. The submission suggests that the student's error is not using f-string formatting and explains the importance of using f-string to correctly insert variables into the string.
3. However, the student's code does use print statements correctly to output the desired format without explicitly using f-string formatting. The issue in the student's code is not the absence of f-string formatting but rather that the function was called twice, leading to duplicate output.
4. The feedback provided in the submission does not accurately identify the mistake in the student's code. Instead, it incorrectly points to the lack of f-string formatting as the problem.
5. For a programming newbie, this feedback might be confusing because it directs attention to an aspect of the code (f-string formatting) that is not the source of the error. The ac

**Note**: Student called function instead of just defining it, gpt is not able to detect this case.

In [307]:
step = 7
data = python['python']['code_problems'][step]
feedback = find_async_feedback_by_id("python", feedbacks, step)
llm_feedback = feedback['response']['output']
passed = feedback['success']
grading = feedback['gradingResult']
render_data(data, llm_feedback, passed, grading)

HTML(value='<h1>Type casting</h1>')

HTML(value="<h2><a href='https://hyperskill.org/learn/step/6224'>Lexical reduplication</a></h2>")

HTML(value='<p></p>\n<p>The languages of the world are amazing! Programming languages too, but now we will tal…

HTML(value='<h3>Wrong Code Submission</h3><pre>word = input()\n\n# Change the next line\nprint(word)</pre>')

HTML(value='<h3>Test Cases</h3>')

...

HTML(value='<h3>GPT New Feedback (Passed: True)</h3><pre>In your code, you are only printing the word once ins…

HTML(value='<h3>Evaluation feedback:</h3>')

HTML(value='<h3>Score: 1. Passed: True</h3>')

HTML(value='<h3>Reason:</h3>')

All assertions passed



**Note**: Output contains direct solution. We have to improve prompt handling this case

In [266]:
step = 8
data = python['python']['code_problems'][step]
feedback = find_async_feedback_by_id("python", feedbacks, step)
llm_feedback = feedback['response']['output']
passed = feedback['success']
grading = feedback['gradingResult']
render_data(data, llm_feedback, passed, grading)

HTML(value="<h1>Taking user's input</h1>")

HTML(value="<h2><a href='https://hyperskill.org/learn/step/5913'>Sum</a></h2>")

HTML(value='<p></p><p>Write a program that reads three numbers from the input separately and prints the sum of…

HTML(value='<h3>Wrong Code Submission</h3><pre>a = int(input)\nb = int(input)\nc = int(input)\nsum = a + b + c…

HTML(value='<h3>Test Cases</h3>')

...

HTML(value="<h3>GPT New Feedback (Passed: False)</h3><pre>In your code, you are trying to convert the input fu…

HTML(value='<h3>Evaluation feedback:</h3>')

HTML(value='<h3>Score: 0.75. Passed: False</h3>')

HTML(value='<h3>Reason:</h3>')

The submission does not meet the criterion:
1. The criterion specifies that the feedback should not give a direct answer.
2. The submission identifies the specific error in the student's code, which is trying to convert the input function directly to an integer.
3. The submission provides actionable advice on how to correct the error by explaining the need to call the input function before converting it to an integer.
4. However, the submission then directly provides the corrected code, which includes the exact lines of code the student should use to fix their program.
5. Providing the corrected code directly contradicts the criterion of not giving a direct answer.

N

N



**Note:** Direct solution. Again.

### First prompt issues exporation results:
- Frequently gives direct solutions
- Not able to detect function calling in one of the user submissions


### Possible ways to improve the prompt:
- Add the IMPORTANT statement to focus the model on not giving the direct solutions

# Testing the second prompt attempt
Lets try to fix the prompt

In [4]:
# Prompt is adapted for promptfoo evaluation system, so we are using {{}} instead of f-strings
prompt = \
f"""This is feedback of the coding testing system. 
Write a human feedback explaining the student what to fix and why theirs solution is wrong.
{create_data_template(extract_important_data_1(data))}
Highlight the specific errors in the student's code.
Offer actionable advice for correcting these errors.
Encourage best coding practices and make the answer simple to understand for a adult programming newbie with no experience.
Make your asnwer scructured, using list with items.
IMPORTANT: DO NOT GIVE DIRECT SOLUTION, JUST A HINT!""" # Adding this line

with open("./prompts/prompt.txt", 'w') as f:
    f.write(prompt)

NameError: name 'create_data_template' is not defined

In [3]:
prompt

NameError: name 'prompt' is not defined

In [269]:
make_eval("./outputs/output_2.json")


┌──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ id       │ submiss… │ task_de… │ test_ca… │ test_co… │ user_wr… │ This is  │
│          │          │          │          │          │          │ feedback │
│          │          │          │          │          │          │ of the   │
│          │          │          │          │          │          │ coding   │
│          │          │          │          │          │          │ testing  │
│          │          │          │          │          │          │ system.  │
│          │          │          │          │          │          │ Write a  │
│          │          │          │          │          │          │ human    │
│          │          │          │          │          │          │ feedback │
│          │          │          │          │          │          │ explain… │
│          │          │          │          │          │          │ the      │
│          │          │          │          │      

» Run promptfoo view to use the local web viewer
» Run promptfoo share to create a shareable URL
» This project needs your feedback. What's one thing we can improve? https://forms.gle/YFLgTe1dKJKNSCsU7
Successes: 16
Failures: 4
Token usage: Total 80288, Prompt 33514, Completion 10985, Cached 5177
Done.


----

## SECOND ATTEMPT TOTAL SCORE: 80%

----

### Checking issues in the second prompt

In [272]:
with open("./outputs/output_2.json", 'r') as f:
    feedbacks2 = json.load(f)

In [273]:
step = 1
data = python['python']['code_problems'][step]
feedback = find_async_feedback_by_id("python", feedbacks2, step)
llm_feedback = feedback['response']['output']
passed = feedback['success']
grading = feedback['gradingResult']
render_data(data, llm_feedback, passed, grading)

HTML(value='<h1>Function, arguments and parameters</h1>')

HTML(value="<h2><a href='https://hyperskill.org/learn/step/7248'>Equation</a></h2>")

HTML(value='<p></p>\n<p>Write a function <code class="java">equation_writing()</code> that will <strong>print<…

HTML(value="<h3>Wrong Code Submission</h3><pre>def equation_writing(a, b, c):\n    print(a, 'x +', b, '=', c)\…

HTML(value='<h3>Test Cases</h3>')

...

HTML(value='<h3>GPT New Feedback (Passed: False)</h3><pre>In your code, you are missing the f-string formattin…

HTML(value='<h3>Evaluation feedback:</h3>')

HTML(value='<h3>Score: 0.25. Passed: False</h3>')

HTML(value='<h3>Reason:</h3>')

The submission does not meet the criterion:
1. The criterion requires the feedback to be easy to understand for an adult programming newbie with no experience.
2. The submission suggests that the student's error is not using f-string formatting and explains the importance of using f-string to correctly insert variables into the string.
3. However, the student's code does use print statements correctly to output the desired format without explicitly using f-string formatting. The issue in the student's code is not the absence of f-string formatting but rather that the function was called twice, leading to duplicate output.
4. The feedback provided in the submission does not accurately identify the mistake in the student's code. Instead, it incorrectly points to the lack of f-string formatting as the problem.
5. For a programming newbie, this feedback might be confusing because it directs attention to an aspect of the code (f-string formatting) that is not the source of the error. The ac

**Note**: Still have problem with function calling

In [312]:
step = 4
data = python['python']['code_problems'][step]
feedback = find_async_feedback_by_id("python", feedbacks2, step)
llm_feedback = feedback['response']['output']
passed = feedback['success']
grading = feedback['gradingResult']
render_data(data, llm_feedback, passed, grading)

HTML(value='<h1>Working with strings: basic methods</h1>')

HTML(value="<h2><a href='https://hyperskill.org/learn/step/6842'>Poster artist</a></h2>")

HTML(value='<p>Imagine that you design film posters for a living. Write a program that prints each film title …

HTML(value='<h3>Wrong Code Submission</h3><pre>print(word.upper())\n\n\n\n</pre>')

HTML(value='<h3>Test Cases</h3>')

...

HTML(value="<h3>GPT New Feedback (Passed: False)</h3><pre>In your code, you are trying to print the uppercase …

HTML(value='<h3>Evaluation feedback:</h3>')

HTML(value='<h3>Score: 0.5. Passed: False</h3>')

HTML(value='<h3>Reason:</h3>')

The submission does not meet the criterion:
1. The task was to write a program that prints each film title in all caps, which implies taking an input and converting it to uppercase.
2. The student's code attempted to print a variable in uppercase without defining it first, leading to a NameError.
3. The feedback correctly identifies the error in the student's code: the use of an undefined variable 'word'.
4. The feedback suggests using the correct variable name 'film_title' that should be defined by capturing user input, which aligns with the correct approach to solving the task.
5. However, the feedback incorrectly assumes that the student defined 'film_title' through the input() function, which is not evident in the student's submission. The student's code snippet does not show any variable definition or input capture, only an attempt to print an undefined variable in uppercase.
6. The feedback provides a hint to fix the issue without giving away the direct solution, which is to ensu

**Note**: Model has problems with reasoning, giving the wrong assumptions how to fix the code

### Second prompt issues exporation results:
- Now it is not giving the direct solutions. Good.
- Model has problems with reasoning, giving the wrong suggestions. Bad.


### Possible ways to improve the prompt:
- Add technical part to model answer template with model reasoning, it can improve the its quality

# Making the third prompt

lets make more complex prompt

In [291]:
prompt = f"""
Submission system output:
{create_data_template(extract_important_data_1(data))}
Based on the provided system feedback, 
perform an internal analysis of the specific errors identified in the student's wrong code submission. 
This internal analysis should summarize the issues without resolving them and is meant for internal
processing only. 
Following the analysis, prepare a user-facing feedback message that explains 
these errors and offers actionable advice without directly solving the problem. 
The output should be structured as follows:

Internal Analysis:
- Identify specific errors based on the error message and the code submitted.
- Assess the implications of these errors on the student's understanding of the programming concepts.

++++++
User-Facing Feedback:
- IMPORTANT: Highlight the specific errors identified in student's code.
- Provide structured, actionable advice on how to correct these errors.
- Encourage best coding practices and explain these practices in a manner simple enough for an adult programming novice.
- Use a list format for clarity and simplicity.
IMPORTANT: Do NOT provide the correct solution directly, only hints and guidance.
IMPORTANT: Follow the answer stucture of the examples


Examples of complete output:

Example 1:
```
Internal Analysis:
- Error: 'NameError' for undefined variable 'word'.
- Issue: The student did not define 'word' before attempting to use it, indicating a misunderstanding of how variables must be declared and used.

++++++
User-Facing Feedback:
1. **Issue Identified:** You attempted to use a variable named 'word' which wasn't defined anywhere in your code, resulting in a 'NameError'.
2. **Actionable Advice:** Start by defining all variables before they are used. For instance, include a line such as 'word = input("Please enter a word: ")' at the beginning of your script.
3. **Best Practice:** Always test your program with different inputs to ensure that all variables are properly initialized and used within their scope. This practice helps avoid common errors and improves the reliability of your code.
```

Example 2:
```
Internal Analysis:
- Error: 'SyntaxError' due to missing colon after an if statement.
- Issue: The student forgot to put a colon at the end of the if statement, which is crucial for defining block scopes in Python.

++++++
User-Facing Feedback:
1. **Issue Identified:** Your if statement is missing a colon at the end, which caused a 'SyntaxError'.
2. **Actionable Advice:** Remember to add a colon right after the condition in your if statements. This tells Python that you're about to start a block of statements that should only execute under certain conditions.
3. **Best Practice:** Writing small test cases to check each part of your code incrementally can help catch these types of syntax errors before they affect larger parts of your program.
```

Example 3:
```
Internal Analysis:
- Error: 'IndexError' because the code attempts to access a list index that is out of range.
- Issue: The student seems unaware of how to properly check the bounds of a list before accessing its elements.

++++++
User-Facing Feedback:
1. **Issue Identified:** You tried to access a list element with an index that exceeds the list's boundaries, leading to an 'IndexError'.
2. **Actionable Advice:** Always check that your index is within the valid range of the list by comparing it against the length of the list minus one. For example, you can use a condition like `if index < len(my_list):` before accessing `my_list[index]`.
3. **Best Practice:** Implementing boundary checks for list accesses is a crucial habit that can prevent runtime errors and ensure your program handles data gracefully.
```
"""

# Prompt is adapted for promptfoo evaluation system, so we are using {{}} instead of f-strings
with open("./prompts/prompt.txt", 'w') as f:
    f.write(prompt)

In [295]:
make_eval("./outputs/output_3.json")

----

## THIRD ATTEMPT TOTAL SCORE: 90%

----

Let's check the bad cases

In [293]:
with open("./outputs/output_3.json", 'r') as f:
    feedbacks3 = json.load(f)

In [311]:
step = 1
data = python['python']['code_problems'][step]
feedback = find_async_feedback_by_id("python", feedbacks3, step)
llm_feedback = feedback['response']['output']
passed = feedback['success']
grading = feedback['gradingResult']
render_data(data, llm_feedback, passed, grading)

HTML(value='<h1>Function, arguments and parameters</h1>')

HTML(value="<h2><a href='https://hyperskill.org/learn/step/7248'>Equation</a></h2>")

HTML(value='<p></p>\n<p>Write a function <code class="java">equation_writing()</code> that will <strong>print<…

HTML(value="<h3>Wrong Code Submission</h3><pre>def equation_writing(a, b, c):\n    print(a, 'x +', b, '=', c)\…

HTML(value='<h3>Test Cases</h3>')

...

HTML(value='<h3>GPT New Feedback (Passed: False)</h3><pre><ol>\n<li><strong>Issue Identified:</strong> Your co…

HTML(value='<h3>Evaluation feedback:</h3>')

HTML(value='<h3>Score: 0.75. Passed: False</h3>')

HTML(value='<h3>Reason:</h3>')

The submission does not meet the criterion:
1. The task was to implement a function that prints an equation in a specific format, taking three arguments.
2. The error identified in the system feedback was that the student's code outputted the correct answer twice instead of once, which is not directly related to the use of f-strings or string concatenation.
3. The internal analysis incorrectly identifies the issue as not utilizing f-strings correctly, which is not the problem indicated by the system feedback. The actual problem seems to be related to the function being called twice or a misunderstanding in how the output is being evaluated.
4. The user-facing feedback focuses on advising the use of f-strings for formatting, which, while being a good practice, does not address the specific error of the output being printed twice.
5. The criterion requires the feedback to highlight specific errors in the student's code and provide actionable advice to correct these errors. The provided f

**Note**: Still have problem with function calling. Need another way to fix it.

In [319]:
step = 6
data = java['java']['code_problems'][step]
feedback = find_async_feedback_by_id("java", feedbacks3, step)
llm_feedback = feedback['response']['output']
passed = feedback['success']
grading = feedback['gradingResult']
render_data(data, llm_feedback, passed, grading)

HTML(value='<h1>Atomics</h1>')

HTML(value="<h2><a href='https://hyperskill.org/learn/step/29304'>Flagging</a></h2>")

HTML(value='<p></p>\n<p>Suppose you have a shared boolean variable represented by an <code class="java">Atomic…

HTML(value='<h3>Wrong Code Submission</h3><pre>import java.util.concurrent.atomic.AtomicBoolean;\n\nclass Flag…

HTML(value='<h3>Test Cases</h3>')

...

HTML(value='<h3>GPT New Feedback (Passed: False)</h3><pre><ol>\n<li><strong>Issue Identified:</strong> Your <c…

HTML(value='<h3>Evaluation feedback:</h3>')

HTML(value='<h3>Score: 0.75. Passed: False</h3>')

HTML(value='<h3>Reason:</h3>')

The submission does not meet the criterion:
The criterion for the feedback is that it must be easy to understand for an adult programming newbie with no experience. Let's break down the user-facing feedback provided to see if it meets this criterion:

1. **Issue Identified:** The feedback clearly identifies the issue in the student's code, stating that the `setFlag` method only returns the current value of the `flag` without updating it. This is straightforward and directly addresses the problem without using overly technical language.

2. **Actionable Advice:** The feedback suggests using the `compareAndSet` method of the `AtomicBoolean` class to atomically set the value of the `flag`. While the term "atomically" might be new to a complete beginner, the advice is actionable because it specifies the method to use. However, the explanation could be slightly intimidating for someone with no experience, as it assumes familiarity with concepts like atomic operations and shared variables.



**Note**: Feedback terminology is too complex for programming newbie. But it is clear and correct.

### Second prompt issues exporation results:
- Still have problem with function calling. Need another approach to fix it.
- Tech part with model reasoning gave the better results
- Sometimes 


### Possible ways to improve the prompt:
- Experiment with IMPORTANT statements to make model focus on the simple terminology in answers

# Conclusion

I think that score of 90% is enough for showing the prompt improvement, so lets finish the improving session.


## What corner cases are not covered by evaluation system now and have to be improved in the future

- Evaluation system has no assertion of all code issues coverage (need manual dataset markup)
- Evaluation system has no assertion of response format (need to determine it with backend developer)

## Why score of 90 is acceptable

- GPT is not fully deterministic, so we have to accept that achiving the 100% score may take a lot of time.
- We have to add more items the requiements list of the prompt asserts to ensure that prompt has good quality and format.

## Results comparation (original and improved)

Lets take good results and compare with the initial model answer:

In [324]:
step = 4
data = python['python']['code_problems'][step]
feedback = find_async_feedback_by_id("python", feedbacks3, step)
llm_feedback = feedback['response']['output']
passed = feedback['success']
grading = feedback['gradingResult']
render_data(data, llm_feedback, passed, grading)

HTML(value='<h1>Working with strings: basic methods</h1>')

HTML(value="<h2><a href='https://hyperskill.org/learn/step/6842'>Poster artist</a></h2>")

HTML(value='<p>Imagine that you design film posters for a living. Write a program that prints each film title …

HTML(value='<h3>Wrong Code Submission</h3><pre>print(word.upper())\n\n\n\n</pre>')

HTML(value='<h3>Test Cases</h3>')

...

HTML(value="<h3>GPT New Feedback (Passed: True)</h3><pre><ol>\n<li><strong>Issue Identified:</strong> The erro…

HTML(value='<h3>Evaluation feedback:</h3>')

HTML(value='<h3>Score: 1. Passed: True</h3>')

HTML(value='<h3>Reason:</h3>')

All assertions passed



**Initial model answer**
1. Define a variable and assign it the input provided.
2. Use the variable with the film title to apply the `upper()` method for converting to uppercase.
3. Ensure the variable defined in step 1 is used inside the `print()` function for the correct output.


----

**New GPT Feedback**

1. **Issue Identified**: The error occurred because you tried to use a variable named 'word' without defining it first, resulting in a 'NameError'.

2. **Actionable Advice**: Always ensure that you define variables before using them in your code. For instance, you can define 'word' by assigning it the value of the input using 'word = input()'.

3. **Best Practice**: To avoid 'NameError' and similar issues, make it a habit to initialize all variables before referencing them in your code. This practice helps in maintaining code clarity and preventing common errors.


----
**Prompt comparation conclusions**

- The new prompt gives more human-like explanation, especially helpfull for newbies
- It gives more common hint, but also gives the best practices advice 

## How to improve the prompt evaluation system
- Add more examples
- Perform manual data markup with feedback and add it to asserts for test cases

## How to make the process of the continious prompt improving

- Once a day, take few samples of the responses from the production
- Evaluate them and make an average scoring metric dashboard
- Monitor it and make improving goals
- Continiosly add examples to the testing dataset
- Collect bad responses examples, add them to asserts to make scoring metric more accurate